## Let's clean our dataset

we need:
- Salary parsing.
- Company name text only.
- State field.
- Age of company.
- Job description parsing

In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('..\datasets\glassdoor_jobs_dataset.csv')
df

Unnamed: 0                                          Job Title  \
0             0                                     Data Scientist   
1             1                          Healthcare Data Scientist   
2             2                                     Data Scientist   
3             3                                     Data Scientist   
4             4                                     Data Scientist   
..          ...                                                ...   
951         951                               Senior Data Engineer   
952         952  Project Scientist - Auton Lab, Robotics Institute   
953         953                               Data Science Manager   
954         954                                      Data Engineer   
955         955          Research Scientist – Security and Privacy   

                 Salary Estimate  \
0     $53K-$91K (Glassdoor est.)   
1    $63K-$112K (Glassdoor est.)   
2     $80K-$90K (Glassdoor est.)   
3     $56K-$97K (Glassdoor est.)   
4    $86K-$143K (Glassdoor est.)   
..                           ...   
951  $72K-$133K (Glassdoor est.)   
952   $56K-$91K (Glassdoor est.)   
953  $95K-$160K (Glassdoor est.)   
954                           -1   
955  $61K-$126K (Glassdoor est.)   

                                       Job Description  Rating  \
0    Data Scientist\r\nLocation: Albuquerque, NM\r\...     3.8   
1    What You Will Do:\r\n\r\nI. General Summary\r\...     3.4   
2    KnowBe4, Inc. is a high growth information sec...     4.8   
3    *Organization and Job ID**\r\nJob ID: 310709\r...     3.8   
4    Data Scientist\r\nAffinity Solutions / Marketi...     2.9   
..                                                 ...     ...   
951  THE CHALLENGE\r\nEventbrite has a world-class ...     4.4   
952  The Auton Lab at Carnegie Mellon University is...     2.6   
953  Data Science ManagerResponsibilities:\r\n\r\nO...     3.2   
954  Loading...\r\n\r\nTitle: Data Engineer\r\n\r\n...     4.8   
955  Returning Candidate? Log back in to the Career...     3.6   

                                     Company Name         Location  \
0                        Tecolote Research\r\n3.8  Albuquerque, NM   
1    University of Maryland Medical System\r\n3.4    Linthicum, MD   
2                                  KnowBe4\r\n4.8   Clearwater, FL   
3                                     PNNL\r\n3.8     Richland, WA   
4                       Affinity Solutions\r\n2.9     New York, NY   
..                                            ...              ...   
951                             Eventbrite\r\n4.4    Nashville, TN   
952         Software Engineering Institute\r\n2.6   Pittsburgh, PA   
953                           Numeric, LLC\r\n3.2    Allentown, PA   
954                                   IGNW\r\n4.8       Austin, TX   
955           Riverside Research Institute\r\n3.6  Beavercreek, OH   

          Headquarters                    Size  Founded  \
0           Goleta, CA   501 to 1000 employees     1973   
1        Baltimore, MD        10000+ employees     1984   
2       Clearwater, FL   501 to 1000 employees     2010   
3         Richland, WA  1001 to 5000 employees     1965   
4         New York, NY     51 to 200 employees     1998   
..                 ...                     ...      ...   
951  San Francisco, CA  1001 to 5000 employees     2006   
952     Pittsburgh, PA   501 to 1000 employees     1984   
953    Chadds Ford, PA       1 to 50 employees       -1   
954       Portland, OR    201 to 500 employees     2015   
955      Arlington, VA   501 to 1000 employees     1967   

          Type of ownership                          Industry  \
0         Company - Private               Aerospace & Defense   
1        Other Organization  Health Care Services & Hospitals   
2         Company - Private                 Security Services   
3                Government                            Energy   
4         Company - Private           Advertising & Marketing  

In [3]:
df.columns.to_list()

['Unnamed: 0',
 'Job Title',
 'Salary Estimate',
 'Job Description',
 'Rating',
 'Company Name',
 'Location',
 'Headquarters',
 'Size',
 'Founded',
 'Type of ownership',
 'Industry',
 'Sector',
 'Revenue',
 'Competitors']

In [4]:
# salary parsing

# detect 'per hour' pattern
df['hourly'] = df['Salary Estimate'].apply(lambda x: 1 if 'per hour' in x.lower() else 0)

# detect employer provided salary:
df['employer_provided'] = df['Salary Estimate'].apply(lambda x: 1 if 'employer provided salary:' in x.lower() else 0)

# remove '-1'
df = df[df['Salary Estimate']!='-1']

# remove '(Glassdoor est.)', 'K', '$' patterns
salary = df['Salary Estimate'].apply(lambda x: x.split('(')[0])
minus_Kd = salary.apply(lambda x: x.replace('K','').replace('$',''))

# remove 'per hour' and 'employer provided salary:' 
min_hr = minus_Kd.apply(lambda x: x.lower().replace('per hour','').replace('employer provided salary:',''))

# create min_salary, max_salary, avg_salary columns
df['min_salary'] = min_hr.apply(lambda x: int(x.split('-')[0]))
df['max_salary'] = min_hr.apply(lambda x: int(x.split('-')[1]))
df['avg_salary'] = (df.min_salary+df.max_salary)/2

<ipython-input-4-cb3db1f28137>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['min_salary'] = min_hr.apply(lambda x: int(x.split('-')[0]))
<ipython-input-4-cb3db1f28137>:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['max_salary'] = min_hr.apply(lambda x: int(x.split('-')[1]))
<ipython-input-4-cb3db1f28137>:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: 

In [5]:
# company name text only
df['company_txt'] = df.apply(lambda x: x['Company Name'] if x['Rating'] <0 else x['Company Name'][:-3], axis = 1)

<ipython-input-5-54dd33da3f41>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['company_txt'] = df.apply(lambda x: x['Company Name'] if x['Rating'] <0 else x['Company Name'][:-3], axis = 1)


In [6]:
# state field 
df['job_state'] = df['Location'].apply(lambda x: x.split(',')[1])
df.job_state.value_counts()
df['same_state'] = df.apply(lambda x: 1 if x.Location == x.Headquarters else 0, axis = 1)

<ipython-input-6-5a0c7b439593>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['job_state'] = df['Location'].apply(lambda x: x.split(',')[1])
<ipython-input-6-5a0c7b439593>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['same_state'] = df.apply(lambda x: 1 if x.Location == x.Headquarters else 0, axis = 1)


In [7]:
# age of company 
df['age'] = df.Founded.apply(lambda x: x if x <1 else 2022 - x)

<ipython-input-7-7582f78e32a2>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['age'] = df.Founded.apply(lambda x: x if x <1 else 2022 - x)


In [8]:
#parsing of job description (python, etc.)

#python
df['python_yn'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)
 
#r studio 
df['R_yn'] = df['Job Description'].apply(lambda x: 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)
df.R_yn.value_counts()

#spark 
df['spark'] = df['Job Description'].apply(lambda x: 1 if 'spark' in x.lower() else 0)
df.spark.value_counts()

#aws 
df['aws'] = df['Job Description'].apply(lambda x: 1 if 'aws' in x.lower() else 0)
df.aws.value_counts()

#excel
df['excel'] = df['Job Description'].apply(lambda x: 1 if 'excel' in x.lower() else 0)
df.excel.value_counts()

df = df.drop(['Unnamed: 0'], axis =1)
df.columns

# df.to_csv('salary_data_cleaned.csv', index = False)

<ipython-input-8-29a7462caf49>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['python_yn'] = df['Job Description'].apply(lambda x: 1 if 'python' in x.lower() else 0)
<ipython-input-8-29a7462caf49>:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['R_yn'] = df['Job Description'].apply(lambda x: 1 if 'r studio' in x.lower() or 'r-studio' in x.lower() else 0)
<ipython-input-8-29a7462caf49>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_

Index(['Job Title', 'Salary Estimate', 'Job Description', 'Rating',
       'Company Name', 'Location', 'Headquarters', 'Size', 'Founded',
       'Type of ownership', 'Industry', 'Sector', 'Revenue', 'Competitors',
       'hourly', 'employer_provided', 'min_salary', 'max_salary', 'avg_salary',
       'company_txt', 'job_state', 'same_state', 'age', 'python_yn', 'R_yn',
       'spark', 'aws', 'excel'],
      dtype='object')